In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('uber_cleaned.csv')
df.head()

,fare_id,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_latitude_rounded,pickup_longitude_rounded,haversine_distance,pickup_year,pickup_month,pickup_day,pickup_hour,pickup_minute,pickup_dayofweek,pickup_dayofweek_label,fare_per_passenger
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1,40.74,-74.00,1.683323,2015,5,7,19,52,3,Thursday,7.500000
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1,40.73,-73.99,2.457590,2009,7,17,20,4,4,Friday,7.700000
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1,40.74,-74.01,5.036377,2009,8,24,21,45,0,Monday,12.900000
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3,40.79,-73.98,1.661683,2009,6,26,8,22,4,Friday,1.766667
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5,40.74,-73.93,4.475450,2014,8,28,17,47,3,Thursday,3.200000


# Questions to answer:
## Business Performance
1. What are the peak demand hours/days?
2. Which locations generate highest revenue per trip?
3. How does fare vary by distance, time, and passenger count?
4. What's the average trip distance and duration proxy?

In [ ]:
import plotly.express as px

# Count trips for each hour and day of week
heatmap_data = df.groupby(['pickup_dayofweek_label', 'pickup_hour']).size().reset_index(name='trip_count')

# Pivot for heatmap
heatmap_pivot = heatmap_data.pivot(index='pickup_dayofweek_label', columns='pickup_hour', values='trip_count')

# Plotly heatmap
fig = px.imshow(
    heatmap_pivot,
    labels=dict(x="Pickup Hour", y="Day of Week", color="Trip Count"),
    x=heatmap_pivot.columns,
    y=heatmap_pivot.index,
    aspect="auto",
    title="Uber Trips Heatmap by Hour and Day of Week"
)
fig.show()

In [10]:
# Calculate total and average revenue per pickup location (rounded coordinates for grouping)
pickup_revenue = df.groupby(['pickup_latitude_rounded', 'pickup_longitude_rounded']).agg(
    total_revenue=('fare_amount', 'sum'),
    avg_revenue=('fare_amount', 'mean'),
    trip_count=('fare_amount', 'count')
).reset_index()

pickup_revenue.sort_values('total_revenue', ascending=False).head(10)

,pickup_latitude_rounded,pickup_longitude_rounded,total_revenue,avg_revenue,trip_count
539,40.76,-73.97,117217.34,9.702619,12081
502,40.75,-73.99,112541.26,10.103354,11139
538,40.76,-73.98,111055.08,10.693797,10385
503,40.75,-73.98,95805.91,9.800114,9776
537,40.76,-73.99,93938.54,10.644594,8825
471,40.74,-73.99,84825.28,9.520233,8910
432,40.73,-73.99,72200.57,9.968324,7243
592,40.77,-73.87,69633.61,29.872849,2331
581,40.77,-73.98,64057.11,9.991750,6411
431,40.73,-74.00,62345.54,9.999285,6235


## Pricing Insights
5. Is there dynamic pricing evidence? (fare/km variance by hour)
6. Are there fare anomalies or outliers to investigate?
7. How does passenger count affect per-passenger pricing?

## Operational Patterns
8. What's the geographic distribution of pickups?
9. Are there seasonal trends in ridership/fares?
10. Which trip categories (short/long) dominate?